In [1]:
import numpy as np

In [2]:
kT = 0.2
dE_0_1 = 0
dE_0_m1 = 0
# The probability of growing to 1
prob_g_1 = np.exp(-(dE_0_1 / kT))
# The probability of growing to -1
prob_g_m1 = np.exp(-(dE_0_m1 / kT))

fit_list = [
    {"bool": dE_0_1 < 0, "prob": prob_g_1, "spin": 1}, 
    {"bool": dE_0_m1 < 0, "prob": prob_g_m1, "spin": -1}            ]
allow_fit = sum([i["bool"] for i in fit_list])

# Both are acceptable.
if allow_fit > 1:
    # Use partition function to find which transition occurs.
    prob_list = [j["prob"] for j in fit_list]
# Only one accepted.
elif allow_fit > 0:
    for j in fit_list:
        if j["bool"]:
            spin = j["spin"]
# Probability of one accepted using detailed balance.
else:
    prob_rand_g = np.random.rand()
    # Use partition function to find which transition occurs.
    allow_prob = sum([prob_rand_g < i["prob"] for i in fit_list])
    if allow_prob > 1:
        prob_list = [j["prob"] for j in fit_list]
    else:
        for j in fit_list:
            if prob_rand_g < j["prob"]:
                spin = j["spin"]

In [9]:
kT = 0.2
dE_0_1 = 0
dE_0_m1 = -1
# The probability of growing to 1
prob_g_1 = np.exp(-(dE_0_1 / kT))
# The probability of growing to -1
prob_g_m1 = np.exp(-(dE_0_m1 / kT))

fit_list = [
    {"bool": dE_0_1 < 0, "prob": prob_g_1, "spin": 1}, 
    {"bool": dE_0_m1 < 0, "prob": prob_g_m1, "spin": -1}            
    ]
allow_fit = sum([i["bool"] for i in fit_list])
bool_list = [i["bool"] for i in fit_list]
for i, j in enumerate(bool_list):
    if j:
        print(bool_list[i])

True
